In [13]:
from statsmodels.discrete.discrete_model import Logit
import pandas as pd 
from patsy import dmatrices

In [29]:
raw_data = pd.read_csv('data/ReplayCharacters 2015-05-14 - 2015-05-24.csv')

In [30]:
raw_data.head()

,ReplayID,Is Auto Select,Hero,Hero Level,Is Winner,MMR Before
0,23915730,False,Rehgar,14,False,3170
1,23915730,False,Falstad,10,False,3330
2,23915730,False,Jaina,7,True,3442
3,23915730,False,Sylvanas,13,False,2666
4,23915730,False,Zagara,10,True,3198


In [31]:
raw_data['present'] = 1

did_win = raw_data['Is Winner'] == True
winners = raw_data.ix[did_win]
losers = raw_data.ix[~did_win]

In [32]:
winners = winners.set_index(['ReplayID', 'Hero'])['present'].unstack().fillna(0)
losers = losers.set_index(['ReplayID', 'Hero'])['present'].unstack().fillna(0)

In [33]:
winners.head(1)

Hero,Abathur,Anub'arak,Arthas,Azmodan,Brightwing,Chen,Diablo,E.T.C.,Falstad,Gazlowe,...,Tassadar,The Lost Vikings,Thrall,Tychus,Tyrael,Tyrande,Uther,Valla,Zagara,Zeratul
ReplayID,,,,,,,,,,,,,,,,,,,,,
20653762,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [34]:
winners['outcome'] = 1
losers['outcome'] = 0 
teams = pd.concat([winners, losers])

In [35]:
teams.shape

(1120496, 37)

In [36]:
formula =  "outcome ~ " +  ' + '.join(map(lambda s: 'Q("%s")'%s.strip(), winners.columns - ['outcome'])) + " -1 "
print formula

outcome ~ Q("Abathur") + Q("Anub'arak") + Q("Arthas") + Q("Azmodan") + Q("Brightwing") + Q("Chen") + Q("Diablo") + Q("E.T.C.") + Q("Falstad") + Q("Gazlowe") + Q("Illidan") + Q("Jaina") + Q("Kael'thas") + Q("Kerrigan") + Q("Li Li") + Q("Malfurion") + Q("Muradin") + Q("Murky") + Q("Nazeebo") + Q("Nova") + Q("Raynor") + Q("Rehgar") + Q("Sgt. Hammer") + Q("Sonya") + Q("Stitches") + Q("Sylvanas") + Q("Tassadar") + Q("The Lost Vikings") + Q("Thrall") + Q("Tychus") + Q("Tyrael") + Q("Tyrande") + Q("Uther") + Q("Valla") + Q("Zagara") + Q("Zeratul") -1 


/Users/camerondavidson-pilon/.virtualenvs/data/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  if __name__ == '__main__':


In [37]:
y, X = dmatrices(formula, teams, return_type='dataframe')

In [39]:
lg = Logit(y, X)
model = lg.fit()


Optimization terminated successfully.
         Current function value: 0.683611
         Iterations 4
                            Results: Logit
Model:               Logit             Pseudo R-squared:  0.014       
Dependent Variable:  outcome           AIC:               1532038.8123
Date:                2016-01-10 20:39  BIC:               1532468.2664
No. Observations:    1120496           Log-Likelihood:    -7.6598e+05 
Df Model:            35                LL-Null:           -7.7667e+05 
Df Residuals:        1120460           LLR p-value:       0.0000      
Converged:           1.0000            Scale:             1.0000      
No. Iterations:      4.0000                                           
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
Q("Abathur")          -0.2089   0.0066 -31.5056 0.0000 -0.2219 -0.1959
Q("

In [44]:
from sklearn.linear_model import LogisticRegression

In [53]:
lg = LogisticRegression()
lg.fit(X,y)

/Users/camerondavidson-pilon/.virtualenvs/data/lib/python2.7/site-packages/sklearn/utils/validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [54]:
lg.coef_

array([[-0.196 ,  0.2824, -0.0923,  0.1414,  0.171 , -0.0828, -0.1933,
        -0.1651,  0.0035,  0.0421,  0.0157,  0.131 ,  0.0711, -0.1237,
         0.1658, -0.119 , -0.1111,  0.2281,  0.1315,  0.0559, -0.0843,
        -0.0054, -0.1397, -0.2291, -0.3443,  0.0981, -0.2173,  0.166 ,
         0.0871, -0.1066,  0.0751, -0.0519,  0.023 , -0.0065,  0.0552,
         0.0008]])

In [57]:
from sklearn.cross_validation import cross_val_score

In [67]:
cross_val_score(lg, X.values, y.values[:,0], cv=10)

array([ 0.5521,  0.5563,  0.5574,  0.5608,  0.5581,  0.5545,  0.5539,
        0.551 ,  0.5545,  0.5532])